**APPLICATION OF DEEP LEARNING IN PREDICTING CURVES**

In [3]:
import warnings

from datetime import datetime

import pandas as pd
import numpy as np
import pandas_datareader.data as web

import seaborn as sns 

import statsmodels.api as sm
from matplotlib import pyplot
from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    cross_val_score,
    train_test_split,
)
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures

warnings.filterwarnings("ignore")

**1. Introduction**

- Yield curve is a graph showing interest rates (bonds) having different maturities. 

**Project Goal:** To find and understand the factors that affect movements pf yield curve. 
- Among the variables are the historical data of the yield curve, federal debt percentage that is held by the investors, and the corporate bond.
- The dependent variable will be the 1-month, 5-year and 30-year tenure of the yield curve.
- Data sources are FRED and Yahoo websites.
- The project uses polynomials of various degrees to forecast the yield curves.

**2. Understanding Data**